In [2]:
import numpy as np
import re
from IPython.display import display, Math, Latex, Image
from PIL import Image
import random
from tqdm import tqdm

# Generate vocabulary and char_to_idx, idx_to_char dictionaries

In [3]:
vocab = open("im2markup/latex_vocab.txt").readlines()
formulae = open("im2markup/formulas.norm.lst",'r').readlines()
char_to_idx = {x.split('\n')[0]:i for i,x in enumerate(vocab)}
# print len(char_to_idx)
char_to_idx['#UNK'] = len(char_to_idx)
char_to_idx['#START'] = len(char_to_idx)
char_to_idx['#END'] = len(char_to_idx)
idx_to_char = {y:x for x,y in char_to_idx.iteritems()}
# print char_to_idx['#UNK']
# print char_to_idx['#START']
# print char_to_idx['#END']
print len(char_to_idx)

502


# Create train, test, validate files

In [17]:
set = "test" # Use train, valid or test to generate corresponding files
file_list = open("im2markup/"+set+".lst",'r').readlines()
set_list = []
missing = {}
for i,line in enumerate(file_list):
    form = formulae[int(line.split()[1])].strip().split()
    out_form = [char_to_idx['#START']]
    for c in form:
        try:
            out_form += [char_to_idx[c]]
        except:
            if c not in missing:
                print c, " not found!"
                missing[c] = 1
            else:
                missing[c] += 1
            out_form += [char_to_idx['#UNK']]
    out_form += [char_to_idx['#END']]
    set_list.append([line.split()[0],out_form])
    
buckets = {}
import os
file_not_found_count = 0
for x,y in tqdm(set_list):
    if os.path.exists('./images_processed/'+x): 
        img_shp = Image.open('./images_processed/'+x).size
        try:
            buckets[img_shp] += [(x,y)]
        except:
            buckets[img_shp] = [(x,y)]
    else:
        file_not_found_count += 1

print "Num files found in %s set: %d/%d"%(set,len(set_list)-file_not_found_count,len(set_list)) 

  0%|          | 0/10355 [00:00<?, ?it/s]

\rightarrowfill  not found!
\verb  not found!
\right/  not found!
\vector  not found!
\bigskip  not found!
\oval  not found!
\mathtt  not found!
\atopwithdelims  not found!
\textcircled  not found!
\scshape  not found!
\symbol  not found!
\uppercase  not found!
\expandafter  not found!
\romannumeral  not found!
\preceq  not found!
\triangleleft  not found!
\]  not found!
\[  not found!
\nulldelimiterspace  not found!
\everymath  not found!
\searrow  not found!
\tabcolsep  not found!
\coprod  not found!
\"  not found!
\nearrow  not found!


100%|██████████| 10355/10355 [00:00<00:00, 24056.33it/s]

Num files found in test set: 9695/10355


In [18]:
print missing
print "Max length of sequence: ",max([len(x[1]) for x in set_list])
## test
print "Testing!"
print buckets[random.choice(buckets.keys())][0]
np.save(set+'_buckets',buckets)

{'\\right/': 3, '\\everymath': 1, '\\searrow': 2, '\\triangleleft': 3, '\\preceq': 1, '\\nearrow': 1, '\\vector': 3, '\\symbol': 1, '\\scshape': 1, '\\"': 1, '\\oval': 2, '\\coprod': 1, '\\nulldelimiterspace': 1, '\\atopwithdelims': 1, '\\textcircled': 1, '\\tabcolsep': 2, '\\]': 1, '\\[': 1, '\\mathtt': 3, '\\verb': 1, '\\rightarrowfill': 1, '\\expandafter': 1, '\\uppercase': 1, '\\bigskip': 3, '\\romannumeral': 1}
Max length of sequence:  627
Testing!
('20032b2645.png', [500, 39, 4, 488, 5, 498, 35, 498, 281, 495, 57, 469, 486, 497, 444, 462, 495, 490, 35, 488, 497, 461, 495, 497, 245, 495, 205, 495, 20, 497, 495, 490, 9, 488, 497, 497, 74, 57, 4, 488, 8, 490, 5, 161, 50, 4, 490, 5, 352, 7, 74, 495, 413, 495, 205, 495, 20, 497, 495, 21, 497, 497, 497, 74, 456, 4, 21, 488, 5, 74, 50, 4, 488, 5, 501])


In [6]:
properties = {}
properties['vocab_size'] = len(vocab)
properties['vocab'] = vocab
properties['char_to_idx'] = char_to_idx
properties['idx_to_char'] = idx_to_char
import numpy as np
np.save('properties',properties)